In [ ]:
!pip install sparseml sparsezoo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 827 kB 26.2 MB/s 
     |████████████████████████████████| 90 kB 8.4 MB/s 
     |████████████████████████████████| 12.3 MB 26.2 MB/s 
     |████████████████████████████████| 97 kB 6.0 MB/s 
     |████████████████████████████████| 1.6 MB 53.4 MB/s 
     |████████████████████████████████| 120 kB 70.9 MB/s 
     |████████████████████████████████| 82 kB 866 kB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=463f85891596b3658b1ecd7fd8cf443ee2ae26f9e421bd1c1a93bccf40f41305
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built GPUtil
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into account 

In [ ]:
!pip install torch==1.9.1+cu111 torchvision==0.10.1+cu111 torchaudio==0.9.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████                   | 834.1 MB 1.5 MB/s eta 0:13:51tcmalloc: large alloc 1147494400 bytes == 0x3a358000 @  0x7f622cd6a615 0x58ead6 0x4f355e 0x4d222f 0x51041f 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x58ff2e 0x50d482 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4bac0a 0x538a76 0x590ae5 0x510280 0x5b4ee6 0x58ff2e 0x50d482 0x5b4ee6 0x58ff2e 0x50c4fc 0x58fd37 0x50ca37 0x5b4ee6 0x58ff2e
     |████████████████▌               | 1055.7 MB 1.4 MB/s eta 0:12:08tcmalloc: large alloc 1434370048 bytes == 0x7e9ae000 @  0x7f622cd6a615 0x58ead6 0x4f355e 0x4d222f 0x51041f 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x58ff2e 0x50d482 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4bac0a 0x538a76 0x590ae5 0x510280 0x5b4ee6 0x58ff2e 0x50d482 0x5b4ee6 0x58ff2e 0x50c4fc 0x58fd37 0x50ca37 0x5b4ee6 0x58ff2e
     |█████

In [ ]:
import sparseml
import sparsezoo
import torch
import torchvision


from sparseml.pytorch.models import ModelRegistry
from sparseml.pytorch.datasets import ImagenetteDataset, ImagenetteSize
from sparseml.pytorch.utils import ModuleExporter

from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from sparseml.pytorch.utils import get_prunable_layers, tensor_sparsity

from tqdm.auto import tqdm
import math
import time
import torch

In [ ]:
print("loading model...")
model = ModelRegistry.create(
    key="resnet50",
    pretrained=True,
    pretrained_dataset="imagenette",
    num_classes=10,
)
input_shape = ModelRegistry.input_shape("resnet50")
input_size = input_shape[-1]

#######################################################
# Define your train and validation datasets below
#######################################################

print("\nloading train dataset...")
train_dataset = ImagenetteDataset(
    train=True, dataset_size=ImagenetteSize.s320, image_size=input_size
)
print(train_dataset)

print("\nloading val dataset...")
val_dataset = ImagenetteDataset(
    train=False, dataset_size=ImagenetteSize.s320, image_size=input_size
)
print(val_dataset)

loading model...


downloading...:   0%|          | 0.00/90.0M [00:00<?, ?B/s]


loading train dataset...


downloading imagenette ImagenetteSize.s320:   0%|          | 0.00/325M [00:00<?, ?B/s]

Dataset ImagenetteDataset
    Number of datapoints: 12894
    Root location: /root/.cache/nm_datasets/imagenette/imagenette-320/train
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=bilinear, max_size=None, antialias=None)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

loading val dataset...
already downloaded imagenette ImagenetteSize.s320
Dataset ImagenetteDataset
    Number of datapoints: 500
    Root location: /root/.cache/nm_datasets/imagenette/imagenette-320/val
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=bilinear, max_size=None, antialias=None)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )


In [ ]:
def run_model_one_epoch(model, data_loader, criterion, device, train=False, optimizer=None):
    if train:
        model.train()
    else:
        model.eval()

    running_loss = 0.0
    total_correct = 0
    total_predictions = 0

    for step, (inputs, labels) in tqdm(enumerate(data_loader), total=len(data_loader)):
        inputs = inputs.to(device)
        labels = labels.to(device)

        if train:
            optimizer.zero_grad()

        outputs, _ = model(inputs)  # model returns logits and softmax as a tuple
        loss = criterion(outputs, labels)

        if train:
            loss.backward()
            optimizer.step()

        running_loss += loss.item()

        predictions = outputs.argmax(dim=1)
        total_correct += torch.sum(predictions == labels).item()
        total_predictions += inputs.size(0)

    loss = running_loss / (step + 1.0)
    accuracy = total_correct / total_predictions
    return loss, accuracy

In [ ]:
# setup device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
print("Using device: {}".format(device))

# setup data loaders
batch_size = 32
train_loader = DataLoader(
    train_dataset, batch_size, shuffle=True, pin_memory=True, num_workers=2
)
val_loader = DataLoader(
    val_dataset, batch_size, shuffle=False, pin_memory=True, num_workers=2
)

# setup loss function and optimizer, LR will be overriden by sparseml
criterion = CrossEntropyLoss()
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)

Using device: cuda


In [ ]:
from sparsezoo import Model, search_models

zoo_model = search_models(
    domain="cv",
    sub_domain="classification",
    architecture="resnet_v1",
    sub_architecture="50",
    framework="pytorch",
    repo="sparseml",
    dataset="imagenette",
    sparse_name="pruned",
)[0]  # unwrap search result
recipe_path = zoo_model.recipes.default.path
print(f"Recipe downloaded to: {recipe_path}")

downloading...:   0%|          | 0.00/4.11k [00:00<?, ?B/s]

Recipe downloaded to: /root/.cache/sparsezoo/d863d8ed-be54-4b9e-8a08-6bdec8219ec5/recipe/recipe_original.md


In [ ]:
from sparseml.pytorch.optim import (
    ScheduledModifierManager,
)

# create ScheduledModifierManager and Optimizer wrapper
manager = ScheduledModifierManager.from_yaml(recipe_path)
optimizer = manager.modify(model, optimizer, steps_per_epoch=len(train_loader))



# Run model pruning
epoch = manager.min_epochs
while epoch < manager.max_epochs:
    # run training loop
    epoch_name = "{}/{}".format(epoch + 1, manager.max_epochs)
    print("Running Training Epoch {}".format(epoch_name))
    train_loss, train_acc = run_model_one_epoch(
        model, train_loader, criterion, device, train=True, optimizer=optimizer
    )
    print(
        "Training Epoch: {}\nTraining Loss: {}\nTop 1 Acc: {}\n".format(
            epoch_name, train_loss, train_acc
        )
    )
    
    # run validation loop
    print("Running Validation Epoch {}".format(epoch_name))
    val_loss, val_acc = run_model_one_epoch(
        model, val_loader, criterion, device
    )
    print(
        "Validation Epoch: {}\nVal Loss: {}\nTop 1 Acc: {}\n".format(
            epoch_name, val_loss, val_acc
        )
    )
    
    epoch += 1

manager.finalize(model)

Running Training Epoch 1/10


  0%|          | 0/403 [00:00<?, ?it/s]

Training Epoch: 1/10
Training Loss: 0.2004798540580329
Top 1 Acc: 0.9395067473243369

Running Validation Epoch 1/10


  0%|          | 0/16 [00:00<?, ?it/s]

Validation Epoch: 1/10
Val Loss: 0.22591056337114424
Top 1 Acc: 0.928

Running Training Epoch 2/10


...

Validation Epoch: 9/10
Val Loss: 0.08900089818052948
Top 1 Acc: 0.976

Running Training Epoch 10/10


  0%|          | 0/403 [00:00<?, ?it/s]

Training Epoch: 10/10
Training Loss: 0.005278394880205412
Top 1 Acc: 0.9992244454785172

Running Validation Epoch 10/10


  0%|          | 0/16 [00:00<?, ?it/s]

Validation Epoch: 10/10
Val Loss: 0.07073573634261265
Top 1 Acc: 0.984



In [ ]:


# print sparsities of each layer
for (name, layer) in get_prunable_layers(model):
    print("{}.weight: {:.4f}".format(name, tensor_sparsity(layer.weight).item()))

input.conv.weight: 0.0000
sections.0.0.conv1.weight: 0.8000
sections.0.0.conv2.weight: 0.8000
sections.0.0.conv3.weight: 0.8000
sections.0.0.identity.conv.weight: 0.8000
sections.0.1.conv1.weight: 0.8000
sections.0.1.conv2.weight: 0.8500
sections.0.1.conv3.weight: 0.8000
sections.0.2.conv1.weight: 0.8000
sections.0.2.conv2.weight: 0.8500
sections.0.2.conv3.weight: 0.8000
sections.1.0.conv1.weight: 0.8000
sections.1.0.conv2.weight: 0.9000
sections.1.0.conv3.weight: 0.8000
sections.1.0.identity.conv.weight: 0.8500
sections.1.1.conv1.weight: 0.8500
sections.1.1.conv2.weight: 0.8500
sections.1.1.conv3.weight: 0.8500
sections.1.2.conv1.weight: 0.8500
sections.1.2.conv2.weight: 0.8500
sections.1.2.conv3.weight: 0.8000
sections.1.3.conv1.weight: 0.8000
sections.1.3.conv2.weight: 0.8500
sections.1.3.conv3.weight: 0.8500
sections.2.0.conv1.weight: 0.8000
sections.2.0.conv2.weight: 0.9000
sections.2.0.conv3.weight: 0.8500
sections.2.0.identity.conv.weight: 0.8500
sections.2.1.conv1.weight: 0.850

In [ ]:
save_dir = "pytorch_classification"

exporter = ModuleExporter(model, output_dir=save_dir)
exporter.export_pytorch(name="resnet50_imagenette_pruned.pth")
exporter.export_onnx(torch.randn(1, 3, 224, 224), name="resnet50_imagenette_pruned.onnx")

In [ ]:
!zip /content/resnet50_sparseml.zip /content/pytorch_classification/resnet50_imagenette_pruned.onnx 

  adding: content/pytorch_classification/resnet50_imagenette_pruned.onnx (deflated 79%)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/resnet50_sparseml.zip /content/drive/MyDrive/

#### Benchmarking both models

In [ ]:
!gdown --id 1-dzX6uC5ad52qDV31LeJ4s8cKlcg_uQw

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1-dzX6uC5ad52qDV31LeJ4s8cKlcg_uQw
To: /content/resnet50_sparseml.zip
100% 20.3M/20.3M [00:00<00:00, 188MB/s]


In [ ]:
!unzip /content/resnet50_sparseml.zip

Archive:  /content/resnet50_sparseml.zip
  inflating: content/pytorch_classification/resnet50_imagenette_pruned.onnx  


In [ ]:
!pip install deepsparse

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 39.0 MB 1.1 MB/s 
     |████████████████████████████████| 97 kB 6.0 MB/s 
     |████████████████████████████████| 90 kB 7.4 MB/s 
     |████████████████████████████████| 13.1 MB 41.2 MB/s 
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires click<8.0,>=5.1, but you have click 8.0.4 which is incompatible.


In [ ]:
!pip install torch==1.9.1+cu111 torchvision==0.10.1+cu111 torchaudio==0.9.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████                   | 834.1 MB 1.4 MB/s eta 0:14:07tcmalloc: large alloc 1147494400 bytes == 0x3a174000 @  0x7fce50d6b615 0x58ead6 0x4f355e 0x4d222f 0x51041f 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x58ff2e 0x50d482 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4bac0a 0x538a76 0x590ae5 0x510280 0x5b4ee6 0x58ff2e 0x50d482 0x5b4ee6 0x58ff2e 0x50c4fc 0x58fd37 0x50ca37 0x5b4ee6 0x58ff2e
     |████████████████▌               | 1055.7 MB 1.4 MB/s eta 0:11:41tcmalloc: large alloc 1434370048 bytes == 0x7e7ca000 @  0x7fce50d6b615 0x58ead6 0x4f355e 0x4d222f 0x51041f 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x58ff2e 0x50d482 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4bac0a 0x538a76 0x590ae5 0x510280 0x5b4ee6 0x58ff2e 0x50d482 0x5b4ee6 0x58ff2e 0x50c4fc 0x58fd37 0x50ca37 0x5b4ee6 0x58ff2e
     |█████

In [ ]:
from torchvision.models import resnet50
import torch
from deepsparse import compile_model
from deepsparse.utils import generate_random_inputs
import time

In [ ]:
batch_size=1
resnet_torch=resnet50(pretrained=True)
resnet_torch=resnet_torch.eval()

engine = compile_model('/content/content/pytorch_classification/resnet50_imagenette_pruned.onnx', batch_size)

dummy_image=torch.randn((1,3,224,224))

inputs = generate_random_inputs('/content/content/pytorch_classification/resnet50_imagenette_pruned.onnx', batch_size)
outputs = engine.run(inputs)
pred=resnet_torch(dummy_image)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

2022-11-01 10:48:41 deepsparse.utils.onnx INFO     Generating input 'input', type = float32, shape = [1, 3, 224, 224]
INFO:deepsparse.utils.onnx:Generating input 'input', type = float32, shape = [1, 3, 224, 224]


In [ ]:
#Benchmark.
t1=time.time()

for _ in range(100):
  pred=resnet_torch(dummy_image)
t2=time.time()

print("Time taken=",(t2-t1))

Time taken= 21.753093719482422


In [ ]:
#Benchmark.
t1=time.time()

for _ in range(100):
  outputs = engine.run(inputs)
t2=time.time()

print("Time taken=",(t2-t1))

Time taken= 9.321174144744873
